## 데이터 전처리 (Data Preprocessing)

In [1]:
import pandas as pd
import numpy as np
from konlpy.tag import Okt
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from collections import Counter

In [2]:
# data 로드하기(train, test), length 출력
# 'ratings_train.txt', ratings_test.txt'
train_data = pd.read_table('./data/ratings_train.txt')
test_data = pd.read_table('./data/ratings_test.txt')

print('train data :',len(train_data)) # 훈련용 리뷰 개수 출력
print('test data :',len(test_data)) # 훈련용 리뷰 개수 출력

train data : 150000
test data : 50000


In [4]:
# 데이터 살펴보기 (컬럼명, 중복항, 라벨 별 count, Null 값 유무)
train_data.head()
# document 중복항 살펴보기: 4000개
train_data['document'].nunique()
# 라벨 별 count 균형 1: 긍 / 0: 부
train_data.groupby('label').size().reset_index(name = 'count')
# Null 값 유무 확인
train_data.isnull().values.any() 

True

In [5]:
# Train data
# Document 컬럼의 중복 제거
train_data.drop_duplicates(subset=['document'], inplace=True)
print(len(train_data))

# Document 컬럼의 Null 값 제거
train_data = train_data.dropna(how = 'any') # Null 값이 존재하는 행 제거
print(train_data.isnull().values.any()) # Null 값이 존재하는지 확인
print(len(train_data))

146183
False
146182


In [6]:
# Document 컬럼의 한글과 공백 텍스트를 제외하고 모두 제거
# 정규식: "[^ㄱ-ㅎㅏ-ㅣ가-힣 ]"
train_data['document'] = train_data['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","", regex=True)
train_data[:5]

,id,document,label
0,9976970,아 더빙 진짜 짜증나네요 목소리,0
1,3819312,흠포스터보고 초딩영화줄오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 솔직히 재미는 없다평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화스파이더맨에서 늙어보이기만 했던 커스틴 던...,1


In [67]:
# # 한글과 공백을 제외하고 모두 제거
# import re
# def remove_special_characters(text):
#     # 한글과 공백을 제외한 모든 문자를 제거
#     return re.sub(r"[^ㄱ-ㅎㅏ-ㅣ가-힣 ]", "", text)
# train_data['document'] = train_data['document'].apply(remove_special_chracters)

In [9]:
# 한글 외 내용을 제거한 뒤 빈문자열이 발생할 수 있음 (영어로만, 특수기호로만 이루어진 문장)
# 공백 데이터 치환 정규식: '^ +'
# 치환 후 Null 값으로 변경
train_data['document'] = train_data['document'].str.replace('^ +', "", regex=True) # white space 데이터를 empty value로 변경
train_data['document'].replace('', np.nan, inplace=True)
print(train_data.isnull().sum())

# Null 값 제거
train_data = train_data.dropna(how = 'any')
print("최종 train 데이터 샘플 수:",len(train_data))

id          0
document    0
label       0
dtype: int64
최종 train 데이터 샘플 수 145393


In [10]:
# Test 데이터 적용
test_data.drop_duplicates(subset = ['document'], inplace=True) # document 열에서 중복인 내용이 있다면 중복 제거
test_data['document'] = test_data['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","", regex=True) # 정규 표현식 수행
test_data['document'] = test_data['document'].str.replace('^ +', "", regex=True) # 공백은 empty 값으로 변경
test_data['document'].replace('', np.nan, inplace=True) # 공백은 Null 값으로 변경
test_data = test_data.dropna(how='any') # Null 값 제거
print("최종 test 데이터 샘플 수:",len(test_data))

최종 test 데이터 샘플 수: 48852


In [17]:
stopwords = ['은', '는', '이', '가', '을', '를', '의', '한', '에', '하', '고', '인', '듯', '과', '와', '네', '들', '듯', '지', '임', '게', '도', '는', '다']

# train 데이터 형태소 분석 및 전처리: 4000개만
X_train = []
okt = Okt()
for sentence in tqdm(train_data['document'][:4000]):
    tokenized_sentence = okt.morphs(sentence, stem=True) # 토큰화
    stopwords_removed_sentence = [word for word in tokenized_sentence if not word in stopwords] # 불용어 제거
    X_train.append(stopwords_removed_sentence)
# 전문 약 15분 소요

100%|██████████| 4000/4000 [00:17<00:00, 222.75it/s]


In [ ]:
# # Mecab 사용을 원할 시 colab 권장
# !git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git
# cd Mecab-ko-for-Google-Colab
# !bash install_mecab-ko_on_colab_light_220429.sh
# from konlpy.tag import Mecab
# mecab = Mecab()
# mecab.morphs("안녕하세요~")

In [18]:
print(X_train[:5])

[['아', '더빙', '진짜', '짜증나다', '목소리'], ['흠', '포스터', '보고', '초딩', '영화', '줄', '오버', '연기', '조차', '가볍다', '않다'], ['너', '무재', '밓었', '다그', '래서', '보다', '추천'], ['교도소', '이야기', '구먼', '솔직하다', '재미', '없다', '평점', '조정'], ['사이', '몬페', '그', '익살스럽다', '연기', '돋보이다', '영화', '스파이더맨', '에서', '늙다', '보이다', '하다', '커스틴', '던스트', '너무나도', '이쁘다', '보이다']]


In [19]:
# test 데이터 형태소 분석 및 전처리: 2000개만
X_test = []
for sentence in tqdm(test_data['document'][:2000]):
    tokenized_sentence = okt.morphs(sentence, stem=True) # 토큰화
    stopwords_removed_sentence = [word for word in tokenized_sentence if not word in stopwords] # 불용어 제거
    X_test.append(stopwords_removed_sentence)

100%|██████████| 2000/2000 [00:08<00:00, 241.17it/s]


In [14]:
# 전처리 된 test 데이터 출력
print(X_test[:5])

[['굳다', 'ㅋ'], ['뭐', '야', '평점', '나쁘다', '않다', '점', '짜다', '리', '더', '더욱', '아니다'], ['지루하다', '않다', '완전', '막장', '돈', '주다', '보기', '에는'], ['만', '아니다', '별', '다섯', '개', '주다', '왜', '로', '나오다', '제', '심기', '불편하다', '하다'], ['음악', '주가', '되다', '최고', '음악', '영화']]


In [21]:
# train 데이터를 train, valid 데이터로 나누기 (8:2)
y_train = np.array(train_data['label'][:4000])
y_test = np.array(test_data['label'][:2000])

X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.2, random_state=0, stratify=y_train)
# stratify: 클래스 비율 조절 (train/valid)

In [26]:
# 데이터 별 클래스 비율 출력하기 (Train, valid, test)
print('train 데이터 클래스 비율')
print(f'부정 = {round(np.sum(y_train==0)/len(y_train) * 100,3)}%')
print(f'긍정 = {round(np.sum(y_train==1)/len(y_train) * 100,3)}%')
print()
print('valid 데이터 클래스 비율')
print(f'부정 = {round(np.sum(y_valid==0)/len(y_valid) * 100,3)}%')
print(f'긍정 = {round(np.sum(y_valid==1)/len(y_valid) * 100,3)}%')
print()
print('test 데이터의 클래스 비율')
print(f'부정 = {round(np.sum(y_test==0)/len(y_test) * 100,3)}%')
print(f'긍정 = {round(np.count_nonzero(y_test)/len(y_test) * 100,3)}%')

train 데이터 클래스 비율
부정 = 50.062%
긍정 = 49.938%

valid 데이터 클래스 비율
부정 = 50.125%
긍정 = 49.875%

test 데이터의 클래스 비율
부정 = 48.75%
긍정 = 51.25%


In [34]:
# 전체 단어수(Unique) 세기 : X_train 기준
word_list = []
for sent in X_train:
    for word in sent:
      word_list.append(word)

word_counts = Counter(word_list) # list, set 해도 동일한 결과 나옴
print('총 단어수 :', len(word_counts))
# print(word_counts)
# print(word_counts['영화'])

총 단어수 : 6715


In [35]:
# 빈도수 기준으로 상위 10개 출력
vocab = sorted(word_counts, key=word_counts.get, reverse=True)
print(vocab[:10])

['영화', '하다', '보다', '없다', '이다', '있다', '좋다', '정말', '재밌다', '너무']


In [39]:
# 단어 수, 등장 빈도 N번 미만 단어 수, 비율 등 계산 및 출력
threshold = 3
total_cnt = len(word_counts) # 단어의 수
rare_cnt = 0 # 등장 빈도수가 threshold보다 작은 단어의 개수를 카운트
total_freq = 0 # 훈련 데이터의 전체 단어 빈도수 총 합
rare_freq = 0 # 등장 빈도수가 threshold보다 작은 단어의 등장 빈도수의 총 합

# 단어와 빈도수의 쌍(pair)을 key와 value로 받는다.
for key, value in word_counts.items():
    total_freq += value

    # 단어의 등장 빈도수가 threshold보다 작으면
    if(value < threshold):
        rare_cnt += 1
        rare_freq += value

print("단어 voca의 크기 :", total_cnt)
print("등장 빈도가 N 이하인 단어의 수:", rare_cnt)
print("전체 단어 중 등장 빈도가 N 이하인 단어의 비율:", (rare_cnt / total_cnt)*100)
print("전체 등장 빈도에서 N 이하인 단어 등장 빈도 비율:", (rare_freq / total_freq)*100)

단어 voca의 크기 : 6715
등장 빈도가 N 이하인 단어의 수: 4799
전체 단어 중 등장 빈도가 N 이하인 단어의 비율: 71.46686522710351
전체 등장 빈도에서 N 이하인 단어 등장 빈도 비율: 16.45401307041616


In [41]:
# vocab에서 빈도수 2이하인 단어는 제거
vocab_size = total_cnt - rare_cnt
vocab = vocab[:vocab_size]
print('총 단어수 :', len(vocab))

총 단어수 : 1916


In [44]:
# 최종 Voca dictionary 구성하기
# Speical token
word_to_index = {}
word_to_index['<PAD>'] = 0
word_to_index['<UNK>'] = 1

# 단어 dictionary 만들기 {pad:0, unk:1, word:2, word:3}
for index, word in enumerate(vocab) :
  word_to_index[word] = index + 2
vocab_size = len(word_to_index)
print('총 단어수 :', vocab_size)
# print(word_to_index['마음'])

총 단어수 : 1918
154


In [45]:
# 정수 인코딩 method: 데이터를 dictionary를 통해 변환
def texts_to_sequences(tokenized_X_data, word_to_index):
  encoded_X_data = []
  for sent in tokenized_X_data:
    index_sequences = []
    for word in sent:
      try:
          index_sequences.append(word_to_index[word])
      except KeyError:
          index_sequences.append(word_to_index['<UNK>'])
    encoded_X_data.append(index_sequences)
  return encoded_X_data

In [46]:
# Train, valid, test 데이터 변환
encoded_X_train = texts_to_sequences(X_train, word_to_index)
encoded_X_valid = texts_to_sequences(X_valid, word_to_index)
encoded_X_test = texts_to_sequences(X_test, word_to_index)

In [48]:
# 인코딩 된 문장 출력해 보기
print(encoded_X_train[1])

[74, 12, 29, 72, 173, 611, 458, 1459, 58, 164, 3, 748, 1, 1460, 128]


In [49]:
# 디코딩 Dictionary 만들기
# word_to_index의 reverse된 dictionary 구성: {idx: words}
index_to_word = {}
for key, value in word_to_index.items():
    index_to_word[value] = key

In [50]:
# 인코딩 된 문장 디코딩 하기, 출력
decoded_sample = [index_to_word[word] for word in encoded_X_train[1]]
print('샘플 :', X_train[1])
print('디코딩 결과 :', decoded_sample)

샘플 : ['주인공', '같다', '사람', '작품', '성', '뛰어나다', '영화인', '지는', '모르다', '이해', '하다', '어렵다', '고저', '한텐', '별로']
디코딩 결과 : ['주인공', '같다', '사람', '작품', '성', '뛰어나다', '영화인', '지는', '모르다', '이해', '하다', '어렵다', '<UNK>', '한텐', '별로']


In [51]:
# padding : 정해진 길이(max_length)에 따라 문서의 길이를 맞춰주는(split or padding) 과정
print('문서의 최대 길이 :', max(len(review) for review in encoded_X_train))
print('문서의 평균 길이 :', sum(map(len, encoded_X_train))/len(encoded_X_train))

문서의 최대 길이 : 54
문서의 평균 길이 : 11.0459375


In [57]:
# 전체 샘플 중 길이가 max_length 이하인 샘플의 비율(X_train 기준)
max_len = 30 # 임의의 max_length 지정
count = 0
for sentence in X_train:
    if(len(sentence) <= max_len):
        count = count + 1
print('전체 샘플 중 길이가 %s 이하인 샘플의 비율: %s'%(max_len, (count / len(X_train))*100))

전체 샘플 중 길이가 30 이하인 샘플의 비율: 93.875


In [58]:
# train, valid, test에 대해 padding 진행: max_length에 맞춰 단어가 없는 구간에 0을 채워 넣는다
def pad_sequences(sentences, max_len):
  features = np.zeros((len(sentences), max_len), dtype=int)
  for index, sentence in enumerate(sentences):
    if len(sentence) != 0:
      features[index, :len(sentence)] = np.array(sentence)[:max_len]
  return features

padded_X_train = pad_sequences(encoded_X_train, max_len=max_len)
padded_X_valid = pad_sequences(encoded_X_valid, max_len=max_len)
padded_X_test = pad_sequences(encoded_X_test, max_len=max_len)

print('train 데이터의 크기 :', padded_X_train.shape)
print('valid 데이터의 크기 :', padded_X_valid.shape)
print('test 데이터의 크기 :', padded_X_test.shape)

훈련 데이터의 크기 : (3200, 30)
검증 데이터의 크기 : (800, 30)
테스트 데이터의 크기 : (2000, 30)


In [60]:
# 패딩 처리 된 데이터 출력해 보기
print('길이 :', len(padded_X_train[0]))
print('샘플 :', padded_X_train[0])

길이 : 30
샘플 : [ 27  46 678 868 404 610   1   3 999   4   1 172   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0]
